# 타입B 태그안뽑기

In [1]:
import re
from tqdm.notebook import tnrange
import csv
import json
from os import walk
import os
import pandas as pd
import datetime

In [2]:
# 파일 불러오기
def get_csv(input_repo, input_filename):
    result = pd.read_csv(f'{input_repo}/{input_filename}')
    
    return result

In [3]:
# 전체 문장에서 태그 안만 뽑음
def save_tag(sentence):
    regex = re.compile('(?<=\<{3}).*?(?=\>{3})')
    result = regex.findall(sentence)
    
    return result

In [4]:
# 전체 문장으로부터 태그 제거
def remove_tag(sentence):
    result = sentence.replace('<<<', '').replace('>>>', '')
        
    return result

In [5]:
# 태그 내 문장으로부터 법률어휘 추출
def draw_word(sentence_tagremoved):
    regex = re.compile('(?<=\<=).*(?=\>)')
    result = regex.search(sentence_tagremoved)
    
    return result

In [6]:
def get_small_tag_removed(sentence):
    keyword_list = save_tag(sentence)
    regex_small_tag = re.compile('(?=\<={1}).*?(?<=\>{1})')
    small_tag_removed = []
    small_tag = []
    for i in range(len(keyword_list)):
        small_tag.append(regex_small_tag.findall(keyword_list[i]))
        small_tag_removed.append(keyword_list[i])

        for j in range(len(small_tag[i])):
            small_tag_removed[i] = small_tag_removed[i].replace(small_tag[i][j], '')
            
    return small_tag_removed

In [7]:
def get_small_tag_removed_throw_morethan2_backup(sentence):
    keyword_list = save_tag(org['transcription'][0])
    regex_small_tag = re.compile('(?=\<={1}).*?(?<=\>{1})')
    small_tag_removed = []
    small_tag_list = []
    for i in range(len(keyword_list)):
        small_tag = regex_small_tag.findall(keyword_list[i])
        if len(small_tag)<2:
            small_tag_list.append(regex_small_tag.search(keyword_list[i]).group())
            small_tag_removed.append(keyword_list[i].replace(small_tag[0], ''))

        else:
            keyword_list[i] = None

    for i in range(len(keyword_list)-1, -1, -1):
        if keyword_list[i] == None:
            del keyword_list[i]

    return small_tag_removed

In [15]:
# def get_small_tag_removed_throw_morethan2(sentence):
keyword_list = save_tag(org['transcription'][0])
regex_small_tag = re.compile('(?=\<={1}).*?(?<=\>{1})')
small_tag_removed = []
small_tag_list = []

for i in range(len(keyword_list)):
    small_tag = regex_small_tag.findall(keyword_list[i])
    if len(small_tag)<2:
        small_tag_list.append(regex_small_tag.search(keyword_list[i]).group())
        small_tag_removed.append(keyword_list[i].replace(small_tag[0], ''))

    else:
        keyword_list[i] = None

for i in range(len(keyword_list)-1, -1, -1):
    if keyword_list[i] == None:
        del keyword_list[i]

#     return small_tag_removed

In [32]:
# def get_small_tag_removed_throw_morethan2_org(sentence):
keyword_list = save_tag(org['original'][0])
regex_small_tag_org = re.compile('(?<=\<={1}).*')

small_tag_removed_org = []
small_tag_list_org = []

for i in range(len(keyword_list)):
    small_tag = regex_small_tag_org.findall(keyword_list[i])
    if len(small_tag)<2:
        small_tag_list_org.append(regex_small_tag_org.search(keyword_list[i]).group())
        small_tag_removed_org.append(keyword_list[i].replace('>', ''))

    else:
        keyword_list[i] = None

for i in range(len(keyword_list)-1, -1, -1):
    if keyword_list[i] == None:
        del keyword_list[i]

#     return small_tag_removed_org

In [34]:
org

,lineNumber,word,originalText,transcription
0,1,호환,"원심판결 이유에 의하면, 원심은 이 사건 행위가 공정거래법 제3조의2 제1항 제3호...","<<<직전의 재판<=원심>판결 이유에>>> 의하면, <<<직전의 재판<=원심>은 이..."
1,2,호환,비록 양 발명의 구성에 상이점이 있어도 그 기술분야에서 통상의 지식을 가진 자가 보...,비록 양 발명의 구성에 <<<서로 다른<=상이> 점이 있어도>>> 그 기술분야에서 ...
2,3,호환,"[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 호환성이 있어 의장법...","[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 <<<연결되는 데 문..."
3,4,호환,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 호환컴퓨터에 대표적으...,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 <<<연결되는 데 문...
4,5,호환,"구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ...","구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ..."
...,...,...,...,...
52432,75991,종용,이에 피해자의 동생은 피고인의 뜻이 확고하다면 이혼을 하더라도 살 집이라도 마련해 ...,이에 피해자의 동생은 <<<형사소송에서 검사에의해 기소당한 사람<=피고인>의 뜻이>...
52433,75992,종용,"[2] 자동차 판매대리점주 甲이 자신의 대리점에서 카마스터(car master, 자...","[2] 자동차 판매대리점주 甲이 자신의 대리점에서 카마스터(car master, 자..."
52434,75993,종용,［2］ 간통죄의 고소인과 그 배우자인 피고소인 쌍방이 혼인관계를 지속할 의사가 없었...,［2］ 간통죄의 고소인과 그 배우자인 <<<민사소송에서 소송을 당한 사람<=피고>소...
52435,75994,종용,"원심은, 원고가 피고보조참가인(이하 ""참가인""이라 한다) 등을 사업장에서 배제하게 ...","<<<직전의 재판<=원심>은, 민사소송을 일으킨 사람<=원고>이>>> <<<민사소송..."


In [33]:
small_tag_removed_org

['직전의 재판<=원심판결 이유에',
 '직전의 재판<=원심은 이',
 '맞거나 틀림<=여부을 판단함에',
 '민사소송에서 소송을 당한 사람<=피고이 별도의',
 '민사소송을 일으킨 사람<=원고에게 연결되는 데 문제가 없음<=호환은 기술의',
 '목적이나 요지<=취지로 설시하였음을',
 '직전의 재판<=원심의 판단은',
 '원심 판결에 불복해 재심사 요청<=상고이유에서 주장하는',
 '잘못된 것을 바로잡는 것<=시정은 명령의 해석에',
 '법률의 근본 원리<=법리 오해의 법령에 어긋남<=위법이']

In [30]:
small_tag

['<=법리>', '<=위법>']

In [27]:
small_tag_list

['<=원심>', '<=원심>', '<=여부>', '<=피고>', '<=취지>', '<=원심>', '<=상고>', '<=시정>']

In [28]:
small_tag_removed

['직전의 재판판결 이유에',
 '직전의 재판은 이',
 '맞거나 틀림을 판단함에',
 '민사소송에서 소송을 당한 사람이 별도의',
 '목적이나 요지로 설시하였음을',
 '직전의 재판의 판단은',
 '원심 판결에 불복해 재심사 요청이유에서 주장하는',
 '잘못된 것을 바로잡는 것은 명령의 해석에']

In [29]:
keyword_list

['직전의 재판<=원심>판결 이유에',
 '직전의 재판<=원심>은 이',
 '맞거나 틀림<=여부>을 판단함에',
 '민사소송에서 소송을 당한 사람<=피고>이 별도의',
 '목적이나 요지<=취지>로 설시하였음을',
 '직전의 재판<=원심>의 판단은',
 '원심 판결에 불복해 재심사 요청<=상고>이유에서 주장하는',
 '잘못된 것을 바로잡는 것<=시정>은 명령의 해석에']

In [16]:
test = regex_small_tag_org.search(keyword_list[0])

In [26]:
test.group().replace('>', '')

'원심판결 이유에'

In [20]:
# csv형태로 저장
def save_as_csv(input_repo, output_repo, result):
    year = str(datetime.datetime.now().year)[2:]
    month = str(datetime.datetime.now().month)
    day = str(datetime.datetime.now().day)
    hour = str(datetime.datetime.now().hour)
    minute = str(datetime.datetime.now().minute)

    print(f'{output_repo}의 경로에 태그안뽑기_{year}{month}{day}_{hour}{minute}.csv의 이름으로 파일 생성이 완료되었습니다.')

    result.to_csv(f'{output_repo}/태그안뽑기_{year}{month}{day}_{hour}{minute}.csv', index=False, encoding='utf-8-sig')

In [8]:
input_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/TypeB태그안뽑기'
output_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/TypeB태그안뽑기/TypeB 최종'

input_filename = '태그안뽑기_211110_1616.csv'

In [9]:
org = get_csv(input_repo, input_filename)

In [10]:
org

,lineNumber,word,originalText,transcription
0,1,호환,"원심판결 이유에 의하면, 원심은 이 사건 행위가 공정거래법 제3조의2 제1항 제3호...","<<<직전의 재판<=원심>판결 이유에>>> 의하면, <<<직전의 재판<=원심>은 이..."
1,2,호환,비록 양 발명의 구성에 상이점이 있어도 그 기술분야에서 통상의 지식을 가진 자가 보...,비록 양 발명의 구성에 <<<서로 다른<=상이> 점이 있어도>>> 그 기술분야에서 ...
2,3,호환,"[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 호환성이 있어 의장법...","[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 <<<연결되는 데 문..."
3,4,호환,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 호환컴퓨터에 대표적으...,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 <<<연결되는 데 문...
4,5,호환,"구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ...","구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ..."
...,...,...,...,...
52432,75991,종용,이에 피해자의 동생은 피고인의 뜻이 확고하다면 이혼을 하더라도 살 집이라도 마련해 ...,이에 피해자의 동생은 <<<형사소송에서 검사에의해 기소당한 사람<=피고인>의 뜻이>...
52433,75992,종용,"[2] 자동차 판매대리점주 甲이 자신의 대리점에서 카마스터(car master, 자...","[2] 자동차 판매대리점주 甲이 자신의 대리점에서 카마스터(car master, 자..."
52434,75993,종용,［2］ 간통죄의 고소인과 그 배우자인 피고소인 쌍방이 혼인관계를 지속할 의사가 없었...,［2］ 간통죄의 고소인과 그 배우자인 <<<민사소송에서 소송을 당한 사람<=피고>소...
52435,75994,종용,"원심은, 원고가 피고보조참가인(이하 ""참가인""이라 한다) 등을 사업장에서 배제하게 ...","<<<직전의 재판<=원심>은, 민사소송을 일으킨 사람<=원고>이>>> <<<민사소송..."


In [12]:
result_transcription = []
for i in range(len(org)):
    result_temp = get_small_tag_removed_throw_morethan2(org['transcription'][i])
    result_transcription.append(result_temp)

In [14]:
len(result_transcription)

52437

In [ ]:
save_as_csv(input_repo, output_repo, result)